<p style="text-align: center;">
    <span style="font-size: xx-large; font-weight: bold; color: red;">
        PROJET 7 :
        Implementez un modèle de scoring
    </span>
</p>  

__Sommaire__

**Partie 1 : Mise en place environnement  MLFlow**
- <a href="#C0">Import des bibliotheques</a>
- <a href="#C1">Environnement  MLFlow</a>

**Partie 2 : Exploration des données**
- <a href="#C2">Fonctions utiles</a>
- <a href="#C3">Chargement des fichiers</a>
- <a>Prétraitements des fichiers</a>
    - <a href="#C4"> Test et validation</a>
    - <a href="#C5"> bureau et bureau_balance</a>
    - <a href="#C6"> previous_application</a>
    - <a href="#C7"> posh_cash_balance</a>
    - <a href="#C8"> installments_payement</a>
    - <a href="#C9"> credit_card_balance</a>
- <a>Feature engineering (conservation des features qui ont moins de 80% de données manquantes)</a>
    - <a href="#C10"> df </a>
    - <a href="#C11"> bureau_agg</a>
    - <a href="#C12">previous_applications_agg </a>
    - <a href="#C13">installments_payments_agg </a>
    - <a href="#C14">credit_card_balance_agg</a>
    - <a href="#C15">home_credit </a>
- <a>Exploration variable cible</a>
    - <a href="#C16">feature target </a>

**Partie 3 : Préparation des données**   



# <span style="color:green; font-weight:bold;">Partie 1 : Mise en place environnement  MLFlow</span>

# <a name="C0"><span style="text-decoration: underline;">Import des bibliothèques</span></a>

In [594]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#pip install mlflow
#pip install dask

# <a name="C1"><span style="text-decoration: underline;">Environnement  MLFlow</span></a>

In [597]:
# Configuration du serveur de suivi MLflow (ici en local)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Spécifiez une expérience MLflow
mlflow.set_experiment("Credit Scoring Model")

#lancer dans le terminal pour visualiser : mlflow ui

PermissionError: [Errno 1] Operation not permitted

# <span style="color:green; font-weight:bold;">Partie 2 : Analyse exploratoire des données</span>

# <a name="C2"><span style="text-decoration: underline;">Fonctions utiles</span></a>

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# <a name="C3"><span style="text-decoration: underline;">Chargement des fichiers</span></a>

In [ ]:
# Charger les fichiers en utilisant dask
sample_submission = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/sample_submission.csv")
home_credit = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/HomeCredit_columns_description.csv", encoding="ISO-8859-1")

# <a name="C4"><span style="text-decoration: underline;">Prétraitement des fichiers de validation et d'entrainement</span></a>

In [ ]:
# Prétraitement des fichiers application_train.csv et application_test.csv sans limitation de lignes
def application_train_test(nan_as_category=False):
    # Lire les données complètes et fusionner
    df = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/application_train.csv")
    test_df = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/application_test.csv")
    print("Échantillons d'entraînement : {}, échantillons de test : {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index(drop=True)
    
    # Optionnel : Supprimer les applications avec CODE_GENDER 'XNA' (ensemble d'entraînement)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Caractéristiques catégorielles avec encodage binaire (0 ou 1 ; deux catégories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
        
    # Caractéristiques catégorielles avec encodage One-Hot
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # Valeurs NaN pour DAYS_EMPLOYED : 365.243 -> NaN
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # Quelques nouvelles caractéristiques simples (pourcentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    del test_df
    gc.collect()
    return df

In [ ]:
# Appel de la fonction pour voir le résultat
df = application_train_test(nan_as_category=True)
df.head()

In [ ]:
print('Training/test data shape: ', df.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(df)
missing_values.head(20)

Dans des travaux ultérieurs, nous utiliserons des modèles tels que XGBoost qui peuvent gérer les valeurs manquantes sans avoir besoin d’imputation.

In [ ]:
# nombre des differents types
df.dtypes.value_counts()

# <a name="C5"><span style="text-decoration: underline;">Prétraitement des données bureau et bureau_balance</span></a>

In [ ]:
# Prétraitement des fichiers bureau.csv et bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    bureau = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/bureau.csv")
    bb = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/bureau_balance.csv")
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance : Effectuer des agrégations et fusionner avec bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()
    
    # Caractéristiques numériques de bureau et bureau_balance
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Caractéristiques catégorielles de bureau et bureau_balance
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau : Crédits actifs - en utilisant uniquement les agrégations numériques
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau : Crédits clôturés - en utilisant uniquement les agrégations numériques
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [ ]:
# Charger et afficher le DataFrame résultant
bureau_agg = bureau_and_balance()

In [ ]:
bureau_agg.head() 

In [ ]:
print('bureau/bureau_balance : ', bureau_agg.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(bureau_agg)
missing_values.head()

# <a name="C5"><span style="text-decoration: underline;">Prétraitement des données de previous_application</span></a>

In [ ]:
# Fonction pour l'encodage One-Hot
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    df = pd.get_dummies(df, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Prétraitement de previous_applications.csv
def previous_applications(nan_as_category=True):
    prev = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/previous_application.csv")
    prev, cat_cols = one_hot_encoder(prev, nan_as_category=True)
    
    # Valeurs de 365.243 jours -> NaN
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    
    # Ajouter une caractéristique : pourcentage de la valeur demandée / valeur reçue
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Caractéristiques numériques des demandes précédentes
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Caractéristiques catégorielles des demandes précédentes
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Demandes précédentes : Demandes approuvées - uniquement les caractéristiques numériques
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Demandes précédentes : Demandes refusées - uniquement les caractéristiques numériques
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [ ]:
# Exécuter la fonction et afficher le DataFrame
previous_applications_agg = previous_applications()

In [ ]:
previous_applications_agg.head()

In [ ]:
print('previous : ', previous_applications_agg.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(previous_applications_agg)
missing_values.head()

# <a name="C6"><span style="text-decoration: underline;">Prétraitement des données de POS_CASH_balance</span></a>

In [ ]:
# Prétraitement de POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=True):
    pos = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/POS_CASH_balance.csv")
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    
    # Caractéristiques
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Compter le nombre de comptes POS cash
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

In [ ]:
# Charger et visualiser le DataFrame
pos_cash_agg = pos_cash()

In [ ]:
pos_cash_agg.head()

In [ ]:
print('pos_cash : ', pos_cash_agg.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(pos_cash_agg)
missing_values.head()

# <a name="C7"><span style="text-decoration: underline;">Prétraitement des données installments_payments</span></a>

In [ ]:
# Fonction pour l'encodage One-Hot
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    df = pd.get_dummies(df, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Prétraitement de installments_payments.csv
def installments_payments(nan_as_category=True):
    # Lire les données de installments_payments
    ins = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/installments_payments.csv")
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    
    # Pourcentage et différence payés dans chaque versement (montant payé et valeur du versement)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Jours de retard et jours avant échéance (pas de valeurs négatives)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Caractéristiques : effectuer des agrégations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Compter le nombre de comptes de versements
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    # Libérer la mémoire
    del ins
    gc.collect()
    
    return ins_agg

In [ ]:
# Charger et visualiser le DataFrame
installments_payments_agg = installments_payments()

In [ ]:
installments_payments_agg.head()

In [ ]:
print('installments payments : ', installments_payments_agg.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(installments_payments_agg)
missing_values.head()

# <a name="C7"><span style="text-decoration: underline;">Prétraitement des données credit_card_balance</span></a>

In [ ]:
# Prétraitement de credit_card_balance.csv
def credit_card_balance(nan_as_category=True):
    cc = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/credit_card_balance.csv")
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # Agrégations générales
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Compter le nombre de lignes de carte de crédit
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    del cc
    gc.collect()
    return cc_agg

In [ ]:
# Charger et visualiser le DataFrame
credit_card_balance_agg = credit_card_balance()
credit_card_balance_agg.head()

In [ ]:
print('credit_card_balance_agg : ', credit_card_balance_agg.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(credit_card_balance_agg)
missing_values.head()

**sample_submission**

In [ ]:
print('sample_submission : ', sample_submission.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(sample_submission)
missing_values.head()

**HomeCredit_columns_description**

In [ ]:
print('home_credit : ', home_credit.shape)

In [ ]:
# Missing values statistics
missing_values = missing_values_table(home_credit)
missing_values.head()

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes df</span></a>

les data frames qui ont des données manquantes : 
- df
- bureau_agg
- previous_applications_agg
- installments_payments_agg
- credit_card_balance_agg
- home_credit

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(df)

# Filtrer pour obtenir uniquement les caractéristiques avec plus de 80 % de valeurs manquantes
missing_values_80 = missing_values[missing_values['% of Total Values'] > 80]

# Afficher les premières lignes des colonnes avec plus de 80 % de valeurs manquantes
print(missing_values_80)

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes bureau_agg</span></a>

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(bureau_agg)

# Filtrer pour obtenir uniquement les caractéristiques avec plus de 80 % de valeurs manquantes
missing_values_80 = missing_values[missing_values['% of Total Values'] > 80]

# Afficher les premières lignes des colonnes avec plus de 80 % de valeurs manquantes
print(missing_values_80)

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes previous_applications_agg</span></a>

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(previous_applications_agg)

# Filtrer pour obtenir les caractéristiques avec plus de 80 % de valeurs manquantes
features_to_drop = missing_values[missing_values['% of Total Values'] > 80].index

# Supprimer les colonnes avec plus de 80 % de valeurs manquantes du DataFrame principal
previous_applications_agg_cleaned = previous_applications_agg.drop(columns=features_to_drop)

# Afficher la forme du DataFrame après la suppression
print("DataFrame initial :", previous_applications_agg.shape)
print("DataFrame après suppression des colonnes avec plus de 80 % de valeurs manquantes :", previous_applications_agg_cleaned.shape)

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes installments_payments_agg</span></a>

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(installments_payments_agg)

# Filtrer pour obtenir uniquement les caractéristiques avec plus de 80 % de valeurs manquantes
missing_values_80 = missing_values[missing_values['% of Total Values'] > 80]

# Afficher les premières lignes des colonnes avec plus de 80 % de valeurs manquantes
print(missing_values_80)

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes credit_card_balance_agg</span></a>

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(credit_card_balance_agg)

# Filtrer pour obtenir uniquement les caractéristiques avec plus de 80 % de valeurs manquantes
missing_values_80 = missing_values[missing_values['% of Total Values'] > 80]

# Afficher les premières lignes des colonnes avec plus de 80 % de valeurs manquantes
print(missing_values_80)

# <a name="C2"><span style="text-decoration: underline;">Traitement données manquantes home_credit</span></a>

In [ ]:
# Calculer les statistiques des valeurs manquantes
missing_values = missing_values_table(home_credit)

# Filtrer pour obtenir uniquement les caractéristiques avec plus de 80 % de valeurs manquantes
missing_values_80 = missing_values[missing_values['% of Total Values'] > 80]

# Afficher les premières lignes des colonnes avec plus de 80 % de valeurs manquantes
print(missing_values_80)

# <a name="C2"><span style="text-decoration: underline;">Exploration variable cible (target)</span></a>

In [ ]:
df['TARGET'].value_counts()

In [ ]:
# Create an anomalous flag column
df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

df['DAYS_EMPLOYED'].plot.hist(title = 'Histogramme de l’emploi par jour');
plt.xlabel('Jours d’emploi');

In [ ]:
# Trouver les corrélations avec la cible et trier
correlations = df.corr()['TARGET'].sort_values()

# Afficher les corrélations
print('Corrélations les plus positives :\n', correlations.tail(15))
print('\nCorrélations les plus négatives :\n', correlations.head(15))

In [ ]:
# Informations d'âge dans un DataFrame séparé
age_data = df[['TARGET', 'DAYS_BIRTH']]

# Prendre la valeur absolue de DAYS_BIRTH pour enlever les valeurs négatives, puis convertir en années
age_data['DAYS_BIRTH'] = age_data['DAYS_BIRTH'].abs()  # Cette ligne prend la valeur absolue
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Créer des intervalles d'âge personnalisés de 5 en 5 ans
bins = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins=bins)

# Afficher les premières lignes
print(age_data.head(10))

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Graphique des tranches d'âge et de la moyenne de la cible sous forme de diagramme à barres
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Étiquettes du graphique
plt.xticks(rotation = 75)
plt.xlabel('Groupe d\'âge (années)')
plt.ylabel('Défaut de remboursement (%)')
plt.title('Défaut de remboursement par groupe d\'âge')

Il y a une tendance claire : les jeunes demandeurs sont plus susceptibles de ne pas rembourser le prêt! Le taux de non-remboursement est supérieur à 10 % pour les trois groupes d’âge les plus jeunes et inférieur à 5 % pour le groupe d’âge le plus âgé.

# <span style="color:green; font-weight:bold;">Partie 3 : Préparation des données pour la modélisation</span>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import ADASYN

# <a name="C2"><span style="text-decoration: underline;">Prétraitement des données</span></a>

In [ ]:
import pandas as pd
from imblearn.over_sampling import ADASYN
from sklearn.utils import resample
from collections import Counter

# Charger le fichier CSV
df = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/application_train.csv")

# Fonction d'encodage one-hot
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Appliquer l'encodage one-hot
df, new_columns = one_hot_encoder(df, nan_as_category=True)

# Vérifier les colonnes disponibles
print("Colonnes disponibles :", df.columns)

# <a name="C2"><span style="text-decoration: underline;">Traitement données train avec/sans réequilibrage</span></a>

In [ ]:
# Étape 1 : DataFrame sans rééquilibrage
df_no_balancing = df.copy()

# Vérifier les proportions dans le jeu déséquilibré
print("Proportions sans rééquilibrage :")
print(df_no_balancing['TARGET'].value_counts())

In [ ]:
# Préparer les données pour ADASYN
X = df.drop(columns=['TARGET'])  # Features
y = df['TARGET']  # Cible

# Vérifier les valeurs manquantes
print("Nombre de valeurs manquantes avant imputation :", X.isnull().sum().sum())

# Imputer les valeurs manquantes
X = X.fillna(0)

# Vérifier les proportions actuelles
print("Proportions avant rééquilibrage :", Counter(y))

In [ ]:
# Calculer le nombre exact d'échantillons pour équilibrer
count_majority = y.value_counts()[0]  # Nombre dans la classe majoritaire
count_minority = y.value_counts()[1]  # Nombre dans la classe minoritaire
samples_needed = count_majority - count_minority  # Nombre d'échantillons nécessaires pour équilibrer

print(f"Classe majoritaire : {count_majority}")
print(f"Classe minoritaire : {count_minority}")
print(f"Échantillons nécessaires pour équilibrer : {samples_needed}")

# Rééquilibrer les données avec ADASYN
adasyn = ADASYN(sampling_strategy={1: count_majority}, random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y)

In [ ]:
# Créer le DataFrame rééquilibré après ADASYN
df_with_balancing = pd.DataFrame(X_resampled, columns=X.columns)
df_with_balancing['TARGET'] = y_resampled

# Séparer les classes majoritaire et minoritaire
df_majority = df_with_balancing[df_with_balancing['TARGET'] == 0]
df_minority = df_with_balancing[df_with_balancing['TARGET'] == 1]

# Sous-échantillonnage de la classe minoritaire
df_minority_balanced = resample(df_minority, 
                                replace=False,  # Pas de duplication
                                n_samples=len(df_majority),  # Taille cible = classe majoritaire
                                random_state=42)

# Fusionner les deux classes pour obtenir un DataFrame équilibré
df_balanced = pd.concat([df_majority, df_minority_balanced])

# Vérifier les proportions finales
print("Proportions finales après ajustement exact :")
print(df_balanced['TARGET'].value_counts())

In [ ]:
# Sauvegarde ou utilisation des deux DataFrames
df_no_balancing.to_csv("/Users/Nelly/Desktop/projet 7/data/dataset_sans_reequilibrage.csv", index=False)
df_balanced.to_csv("/Users/Nelly/Desktop/projet 7/data/dataset_avec_reequilibrage.csv", index=False)

print("Les fichiers ont été sauvegardés avec succès.")

# <a name="C2"><span style="text-decoration: underline;">Modélisation</span></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
# Charger les deux fichiers
df_no_balancing = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/dataset_sans_reequilibrage.csv")
df_balanced = pd.read_csv("/Users/Nelly/Desktop/projet 7/data/dataset_avec_reequilibrage.csv")

# Préparer les features (X) et la cible (y)
def prepare_data(df):
    X = df.drop(columns=['TARGET'])
    y = df['TARGET']
    return X, y

X_no_balancing, y_no_balancing = prepare_data(df_no_balancing)
X_balanced, y_balanced = prepare_data(df_balanced)

# Diviser les jeux de données en entraînement et test
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(X_no_balancing, y_no_balancing, test_size=0.3, random_state=42, stratify=y_no_balancing)
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42, stratify=y_balanced)

In [ ]:
# Calcul des valeurs manquantes pour chaque dataset
missing_no = X_no_balancing.isnull().sum()  # Somme des valeurs manquantes par colonne
missing_bal = X_balanced.isnull().sum()    # Somme des valeurs manquantes par colonne

# Proportion des données manquantes en pourcentage
missing_no_percent = (missing_no / len(X_no_balancing)) * 100
missing_bal_percent = (missing_bal / len(X_balanced)) * 100

print("Pourcentage des données manquantes (sans rééquilibrage) :")
print(missing_no_percent[missing_no_percent > 0])

print("\nPourcentage des données manquantes (avec rééquilibrage) :")
print(missing_bal_percent[missing_bal_percent > 0])

In [ ]:
import matplotlib.pyplot as plt

# Visualisation des colonnes avec données manquantes
def plot_missing_data(missing_data, title):
    filtered_data = missing_data[missing_data > 0]
    if filtered_data.empty:
        print(f"Aucune donnée manquante pour {title}.")
        return
    filtered_data.sort_values(ascending=False).plot(
        kind='bar', figsize=(12, 6), color="skyblue"
    )
    plt.title(title)
    plt.ylabel("Pourcentage de données manquantes")
    plt.show()

# Utilisation de la fonction corrigée
print("Graphique des données manquantes (sans rééquilibrage) :")
plot_missing_data(missing_no_percent, "Données manquantes (sans rééquilibrage)")

print("Graphique des données manquantes (avec rééquilibrage) :")
plot_missing_data(missing_bal_percent, "Données manquantes (avec rééquilibrage)")

In [ ]:
# Suppression des colonnes avec plus de 50% de données manquantes
high_missing_cols = missing_no_percent[missing_no_percent > 50].index
X_no_balancing = X_no_balancing.drop(columns=high_missing_cols)
X_balanced = X_balanced.drop(columns=high_missing_cols)

# Supprimer les colonnes complètement vides
X_no_balancing = X_no_balancing.dropna(axis=1, how='all')
X_balanced = X_balanced.dropna(axis=1, how='all')

# Vérification des données manquantes
print("Données manquantes dans X_no_balancing :", X_no_balancing.isnull().sum().sum())
print("Données manquantes dans X_balanced :", X_balanced.isnull().sum().sum())

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import lightgbm as lgb
import pandas as pd
import re

In [ ]:
from sklearn.model_selection import train_test_split

# Fonction pour calculer le coût métier
def calculate_business_cost(y_true, y_pred, fn_cost=10, fp_cost=1):
    """
    Calcule le coût métier basé sur les faux négatifs (FN) et faux positifs (FP).
    """
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    business_cost = fn * fn_cost + fp * fp_cost
    return business_cost

# Diviser les données sans rééquilibrage
X_train_no, X_test_no, y_train_no, y_test_no = train_test_split(
    X_no_balancing, y_no_balancing, test_size=0.3, random_state=42, stratify=y_no_balancing
)

# Diviser les données avec rééquilibrage
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(
    X_balanced, y_balanced, test_size=0.3, random_state=42, stratify=y_balanced
)

# Définir les jeux de données
datasets = {
    "Sans rééquilibrage": (X_train_no, X_test_no, y_train_no, y_test_no),
    "Avec rééquilibrage": (X_train_bal, X_test_bal, y_train_bal, y_test_bal)
}

In [ ]:
import mlflow

# Configurez MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Credit Scoring Model")

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def objective(params, model_type, X_train, X_test, y_train, y_test):
    print("Début de la fonction objective")

    # Démarrer un run MLflow
    with mlflow.start_run(run_name=f"{model_type} optimization"):
        print("Run MLflow démarré")
        active_run = mlflow.active_run()
        if active_run:
            print(f"Run ID actif : {active_run.info.run_id}")
        else:
            print("Erreur : Aucun run actif détecté !")

        # Nettoyage des colonnes
        print("Nettoyage des colonnes...")
        X_train = X_train.rename(columns=lambda x: re.sub(r'[^A-Za-z0-9_]', '_', x))
        X_test = X_test.rename(columns=lambda x: re.sub(r'[^A-Za-z0-9_]', '_', x))

        # Initialisation du modèle
        print(f"Initialisation du modèle {model_type}...")
        if model_type == "LightGBM":
            model = lgb.LGBMClassifier(
                n_estimators=int(params.get('n_estimators', 100)),
                num_leaves=int(params.get('num_leaves', 31)),
                learning_rate=params.get('learning_rate', 0.1),
                max_depth=int(params.get('max_depth', -1)),
                subsample=params.get('subsample', 1.0),
                colsample_bytree=params.get('colsample_bytree', 1.0),
                random_state=42
            )
        elif model_type == "RandomForest":
            model = RandomForestClassifier(
                n_estimators=int(params.get('n_estimators', 100)),
                max_depth=int(params.get('max_depth', None)),
                min_samples_split=params.get('min_samples_split', 2),
                min_samples_leaf=params.get('min_samples_leaf', 1),
                random_state=42
            )
        elif model_type == "LogisticRegression":
            model = LogisticRegression(
                C=params.get('C', 1.0),
                solver=params.get('solver', 'lbfgs'),
                max_iter=1000,
                random_state=42
            )
        else:
            raise ValueError("Type de modèle non supporté : {}".format(model_type))

        print("Entraînement du modèle...")
        model.fit(X_train, y_train)

        print("Calcul des métriques...")
        y_proba = model.predict_proba(X_test)[:, 1]
        y_pred = (y_proba >= 0.5).astype(int)

        # Calcul des métriques pertinentes
        auc = roc_auc_score(y_test, y_proba)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        # Calcul du coût métier
        business_cost = calculate_business_cost(y_test, y_pred)

        # Log des métriques dans MLflow
        mlflow.log_metric("AUC", auc)
        mlflow.log_metric("Accuracy", accuracy)
        mlflow.log_metric("Precision", precision)
        mlflow.log_metric("Recall", recall)
        mlflow.log_metric("F1-Score", f1)
        mlflow.log_metric("Business Cost", business_cost)

        print(f"Métriques loguées : AUC={auc}, Accuracy={accuracy}, Precision={precision}, Recall={recall}, F1-Score={f1}, Business Cost={business_cost}")

        # Enregistrement du modèle dans MLflow
        mlflow.sklearn.log_model(model, f"{model_type}_Model")

        print(f"Run ID final pour le Model Registry : {mlflow.active_run().info.run_id}")

        return {
            'loss': -auc,  # AUC négatif pour minimisation
            'status': STATUS_OK,
            'auc': auc,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'business_cost': business_cost
        }

In [ ]:
# Espace des hyperparamètres
param_space_lgb = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'num_leaves': hp.quniform('num_leaves', 20, 50, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth': hp.quniform('max_depth', -1, 10, 1),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)
}


In [ ]:
# Définir les jeux de données
datasets = {
    "Sans rééquilibrage": (X_train_no, X_test_no, y_train_no, y_test_no),
    "Avec rééquilibrage": (X_train_bal, X_test_bal, y_train_bal, y_test_bal)
}

# Tester chaque jeu de données
for name, (X_train, X_test, y_train, y_test) in datasets.items():
    print(f"Optimisation pour le dataset : {name}")
    
    trials_lgb = Trials()
    best_params_lgb = fmin(
        fn=lambda params: objective(params, "LightGBM", X_train, X_test, y_train, y_test),
        space=param_space_lgb,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials_lgb
    )
    
    print(f"Meilleurs hyperparamètres pour {name} :", best_params_lgb)

In [ ]:
# Optimisation des hyperparamètres pour Random Forest
print("Optimisation des hyperparamètres pour Random Forest...")

param_space_rf = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.1, 0.5)
}

trials_rf = Trials()
best_params_rf = fmin(
    fn=lambda params: objective(params, "RandomForest", X_train, X_test, y_train, y_test),
    space=param_space_rf,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials_rf
)

print("Meilleurs hyperparamètres pour Random Forest :", best_params_rf)

In [ ]:
# Optimisation des hyperparamètres pour Logistic Regression
print("Optimisation des hyperparamètres pour Logistic Regression...")

param_space_lr = {
    'C': hp.loguniform('C', -4, 2),  # Exponentielle pour couvrir une large plage
    'solver': hp.choice('solver', ['lbfgs', 'liblinear', 'saga'])
}

trials_lr = Trials()
best_params_lr = fmin(
    fn=lambda params: objective(params, "LogisticRegression", X_train, X_test, y_train, y_test),
    space=param_space_lr,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials_lr
)

print("Meilleurs hyperparamètres pour Logistic Regression :", best_params_lr)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.impute import SimpleImputer
# Fonction pour évaluer un modèle optimisé
def evaluate_optimized_model(model, X_train, X_test, y_train, y_test, fn_cost=10, fp_cost=1):
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]

    best_threshold = 0.5
    best_cost = float('inf')
    results_by_threshold = []

    thresholds = np.arange(0.01, 1.0, 0.01)
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        cost = calculate_business_cost(y_test, y_pred, fn_cost, fp_cost)
        if cost < best_cost:
            best_cost = cost
            best_threshold = threshold

        auc = roc_auc_score(y_test, y_proba)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)

        results_by_threshold.append({
            "Threshold": threshold,
            "AUC": auc,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "Business Cost": cost
        })

    final_metrics = {
        "Optimal Threshold": best_threshold,
        "Optimal Business Cost": best_cost,
        "Results by Threshold": results_by_threshold
    }
    return final_metrics

# Imputation des données pour gérer les NaN
imputer = SimpleImputer(strategy='mean')
X_train_no = imputer.fit_transform(X_train_no)
X_test_no = imputer.transform(X_test_no)

X_train_bal = imputer.fit_transform(X_train_bal)
X_test_bal = imputer.transform(X_test_bal)

# Mise à l'échelle des données
scaler = StandardScaler()
X_train_no_scaled = scaler.fit_transform(X_train_no)
X_test_no_scaled = scaler.transform(X_test_no)

X_train_bal_scaled = scaler.fit_transform(X_train_bal)
X_test_bal_scaled = scaler.transform(X_test_bal)

# Initialisation des résultats
optimized_results_no = {}
optimized_results_bal = {}

# Évaluation des modèles sans rééquilibrage
print("Évaluation sur les données SANS rééquilibrage :")
best_model_lgb_no = lgb.LGBMClassifier(
    n_estimators=int(best_params_lgb['n_estimators']),
    num_leaves=int(best_params_lgb['num_leaves']),
    learning_rate=best_params_lgb['learning_rate'],
    max_depth=int(best_params_lgb['max_depth']),
    subsample=best_params_lgb['subsample'],
    colsample_bytree=best_params_lgb['colsample_bytree'],
    random_state=42
)
optimized_results_no["LightGBM"] = evaluate_optimized_model(best_model_lgb_no, X_train_no, X_test_no, y_train_no, y_test_no)

best_model_rf_no = RandomForestClassifier(
    n_estimators=int(best_params_rf['n_estimators']),
    max_depth=int(best_params_rf['max_depth']),
    min_samples_split=best_params_rf['min_samples_split'],
    min_samples_leaf=best_params_rf['min_samples_leaf'],
    random_state=42
)
optimized_results_no["Random Forest"] = evaluate_optimized_model(best_model_rf_no, X_train_no, X_test_no, y_train_no, y_test_no)

best_model_lr_no = LogisticRegression(
    C=best_params_lr['C'],
    solver=['liblinear', 'lbfgs'][best_params_lr['solver']],
    max_iter=1000,
    random_state=42
)
optimized_results_no["Logistic Regression"] = evaluate_optimized_model(
    best_model_lr_no, X_train_no_scaled, X_test_no_scaled, y_train_no, y_test_no
)

# Évaluation des modèles avec rééquilibrage
print("\nÉvaluation sur les données AVEC rééquilibrage :")
best_model_lgb_bal = lgb.LGBMClassifier(
    n_estimators=int(best_params_lgb['n_estimators']),
    num_leaves=int(best_params_lgb['num_leaves']),
    learning_rate=best_params_lgb['learning_rate'],
    max_depth=int(best_params_lgb['max_depth']),
    subsample=best_params_lgb['subsample'],
    colsample_bytree=best_params_lgb['colsample_bytree'],
    random_state=42
)
optimized_results_bal["LightGBM"] = evaluate_optimized_model(best_model_lgb_bal, X_train_bal, X_test_bal, y_train_bal, y_test_bal)

best_model_rf_bal = RandomForestClassifier(
    n_estimators=int(best_params_rf['n_estimators']),
    max_depth=int(best_params_rf['max_depth']),
    min_samples_split=best_params_rf['min_samples_split'],
    min_samples_leaf=best_params_rf['min_samples_leaf'],
    random_state=42
)
optimized_results_bal["Random Forest"] = evaluate_optimized_model(best_model_rf_bal, X_train_bal, X_test_bal, y_train_bal, y_test_bal)

best_model_lr_bal = LogisticRegression(
    C=best_params_lr['C'],
    solver=['liblinear', 'lbfgs'][best_params_lr['solver']],
    max_iter=1000,
    random_state=42
)
optimized_results_bal["Logistic Regression"] = evaluate_optimized_model(
    best_model_lr_bal, X_train_bal_scaled, X_test_bal_scaled, y_train_bal, y_test_bal
)

In [ ]:
import pandas as pd

def display_results(results, title):
    """
    Affiche les résultats de manière condensée pour chaque modèle, avec des espaces entre les sections.
    """
    print(title)
    summary_results = []

    for model_name, metrics in results.items():
        print(f"\nModèle : {model_name}")
        print(f"Seuil Optimal : {metrics['Optimal Threshold']:.2f}")
        print(f"Coût Métier Optimal : {metrics['Optimal Business Cost']:.2f}\n")  # Ajout d'un espace ici
        
        # Résumé au seuil optimal
        optimal_metrics = next((res for res in metrics["Results by Threshold"] 
                                if res["Threshold"] == metrics["Optimal Threshold"]), None)
        if optimal_metrics:
            summary_results.append({
                "Model": model_name,
                "AUC": optimal_metrics["AUC"],
                "Accuracy": optimal_metrics["Accuracy"],
                "Precision": optimal_metrics["Precision"],
                "Recall": optimal_metrics["Recall"],
                "F1-Score": optimal_metrics["F1-Score"],
                "Business Cost": optimal_metrics["Business Cost"],
                "Optimal Threshold": optimal_metrics["Threshold"]
            })
    
    # Afficher les résultats sous forme de tableau
    results_df = pd.DataFrame(summary_results)
    print(results_df.to_string(index=False))  # Le tableau est affiché ici avec un espace au-dessus

In [ ]:
# Affichage des résultats
display_results(optimized_results_no, "Résultats pour les modèles optimisés SANS rééquilibrage :")
print("\n")  # Ajouter un espace pour séparer les deux blocs de résultats
display_results(optimized_results_bal, "Résultats pour les modèles optimisés AVEC rééquilibrage :")

# Analyse des résultats

## Modèles optimisés SANS rééquilibrage des données

### Modèle : LightGBM
- **Seuil Optimal** : 0.09  
- **Coût Métier Optimal** : 48,908.00  

**Analyse** :  
LightGBM montre une performance correcte avec une AUC de 0.759 et une accuracy de 73.2 %. Toutefois, la précision est faible (0.177), ce qui indique un grand nombre de faux positifs. Le rappel (0.639) est acceptable, mais le coût métier optimal reste élevé.

### Modèle : Random Forest
- **Seuil Optimal** : 0.09  
- **Coût Métier Optimal** : 56,132.00  

**Analyse** :  
Random Forest a des performances légèrement inférieures à celles de LightGBM en termes d'AUC (0.702) et d'accuracy (75.5 %). La précision (0.164) est faible, et le rappel est proche de zéro (0.500), ce qui reflète une incapacité à bien identifier les vrais positifs. Le coût métier est également plus élevé.

### Modèle : Logistic Regression
- **Seuil Optimal** : 0.09  
- **Coût Métier Optimal** : 50,478.00  

**Analyse** :  
La régression logistique présente une AUC légèrement supérieure (0.744) à Random Forest, mais toujours inférieure à LightGBM. L'accuracy est comparable (72.1 %), mais la précision (0.169) et le rappel (0.630) restent faibles. Le coût métier est un peu plus bas que Random Forest.

---

## Modèles optimisés AVEC rééquilibrage des données

### Modèle : LightGBM
- **Seuil Optimal** : 0.10  
- **Coût Métier Optimal** : 48,899.00  

**Analyse** :  
Après rééquilibrage des données, LightGBM améliore considérablement ses performances : AUC de 0.979 et accuracy de 85.3 %. La précision (0.791) et le rappel (0.968) sont particulièrement élevés, ce qui reflète un excellent compromis entre faux positifs et faux négatifs. Le coût métier est également minimal, ce qui en fait le modèle le plus performant.

### Modèle : Random Forest
- **Seuil Optimal** : 0.37  
- **Coût Métier Optimal** : 75,952.00  

**Analyse** :  
Le rééquilibrage améliore les performances de Random Forest, mais elles restent inférieures à celles de LightGBM. L'AUC passe à 0.990, mais la précision (0.557) et le rappel (0.989) restent en deçà des attentes. Le coût métier est significativement plus élevé, ce qui rend ce modèle moins compétitif.

### Modèle : Logistic Regression
- **Seuil Optimal** : 0.08  
- **Coût Métier Optimal** : 52,310.00  

**Analyse** :  
La régression logistique bénéficie également du rééquilibrage. L'AUC atteint 0.976, l'accuracy est de 81.9 %, et la précision est bonne (0.745). Cependant, le rappel (0.971) et le coût métier (52,310.00) indiquent une performance légèrement inférieure à LightGBM.

---

## Conclusion 

### **Meilleur modèle global : LightGBM **



# <span style="color:green; font-weight:bold;">Partie 4 : Analyse des features du meilleur modèle </span>

In [ ]:
import os
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
from contextlib import contextmanager
import time

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print(f"{title} - terminé en {time.time() - t0:.2f}s")

def analyse_shap(model_file, train_df, feats, output_dir="shap_results"):
    """
    Analyse SHAP avec un modèle LightGBM déjà entraîné.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Chargement du modèle LightGBM pour SHAP...")
    clf = lgb.Booster(model_file=model_file)

    print("Préparation des données pour SHAP...")
    data_sample = train_df.sample(10000, random_state=42)[feats]
    print(f"Échantillon : {data_sample.shape}")

    print("Calcul des valeurs SHAP...")
    explainer = shap.TreeExplainer(clf)
    shap_values = explainer.shap_values(data_sample)

    # Vérification pour modèles binaires
    if isinstance(shap_values, list) and len(shap_values) > 1:
        shap_values = shap_values[1]

    print("Création des graphiques SHAP...")
    # Résumé SHAP (violin plot)
    plt.figure()
    shap.summary_plot(shap_values, data_sample, plot_type="violin", show=False)
    plt.savefig(os.path.join(output_dir, "shap_summary_violin.png"))
    plt.show()  # Affiche le graphique dans le notebook
    plt.close()

    # Résumé SHAP (bar plot)
    plt.figure()
    shap.summary_plot(shap_values, data_sample, plot_type="bar", show=False)
    plt.savefig(os.path.join(output_dir, "shap_summary_bar.png"))
    plt.show()  # Affiche le graphique dans le notebook
    plt.close()

    # Force plot pour un client spécifique
    print("Création du graphique SHAP pour un individu (force plot)...")
    sample_idx = 35  # Vous pouvez changer cet indice pour un autre client
    
    # Vérifier si expected_value est une liste ou une valeur scalaire
    expected_value = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
    
    # Générer le force plot
    shap.force_plot(
        expected_value,  # Utiliser la valeur attendue appropriée
        shap_values[sample_idx, :],  # Valeurs SHAP pour l'individu
        data_sample.iloc[sample_idx, :],  # Données de l'individu
        matplotlib=True
    )
    plt.savefig(os.path.join(output_dir, f"shap_force_plot_client_{sample_idx}.png"))
    plt.show()

def main(debug=False):
    print("Chargement des données...")
    df = application_train_test(10000 if debug else None)  # Remplacez par votre fonction de chargement
    df.columns = df.columns.str.replace(r'[^\w\s]', '', regex=True).str.replace(' ', '_')

    # Séparer les données d'entraînement
    train_df = df[df['TARGET'].notnull()]
    feats = [f for f in train_df.columns if f not in ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index']]

    # Charger un modèle déjà entraîné
    model_file = "lightgbm_model_final.txt"
    if not os.path.exists(model_file):
        print(f"Erreur : le fichier du modèle '{model_file}' est introuvable. Veuillez entraîner et sauvegarder le modèle au préalable.")
        return

    # Analyse SHAP
    with timer("Analyse SHAP"):
        analyse_shap(model_file, train_df, feats)

if __name__ == "__main__":
    with timer("Exécution complète"):
        main()

# <span style="color:green; font-weight:bold;">Partie 5 : Data drift </span>

In [ ]:
import os
import pandas as pd
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
import lightgbm as lgb

# Charger les datasets
df_preprocessed = application_train_test(nan_as_category=True)

# Séparer les données d'entraînement (TARGET non nul)
train_df = df_preprocessed[df_preprocessed['TARGET'].notnull()]

# Charger les données de test
test_df = df_preprocessed[df_preprocessed['TARGET'].isnull()]

# Charger le modèle LightGBM
model_file = "lightgbm_model_final.txt"
if not os.path.exists(model_file):
    raise FileNotFoundError(f"Le fichier de modèle '{model_file}' est introuvable.")

clf = lgb.Booster(model_file=model_file)

# Extraire les features importantes à partir du modèle
feature_importances = pd.DataFrame({
    'feature': clf.feature_name(),
    'importance': clf.feature_importance(importance_type='gain')
}).sort_values(by='importance', ascending=False)

# Sélectionner les 20 features les plus importantes
top_features = feature_importances['feature'].head(20).tolist()

# Filtrer les datasets sur les colonnes importantes
train_df_filtered = train_df[available_features]
test_df_filtered = test_df[available_features]

# Création d'un rapport Evidently pour le Data Drift
data_drift_report = Report(metrics=[DataDriftPreset()])

# Calculer le Data Drift entre les données d'entraînement et de test
data_drift_report.run(reference_data=train_df_filtered, current_data=test_df_filtered)

# Sauvegarder le rapport HTML
output_file = "data_drift_report_top_features.html"
data_drift_report.save_html(output_file)

print(f"Le rapport de Data Drift a été généré : {output_file}")